In [1]:
from pytz import timezone
import matplotlib.pyplot as plt
import pandas as pd
import time
import datetime as dt
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz
import plotly.graph_objects as go


utc_tz = timezone('UTC')
pd.set_option('display.max_columns', 500) # number of columns to be displayed
pd.set_option('display.width', 1500)      # max table width to display

In [2]:
# This will need to be updated as it should come directly from Currency Strength Script
MS = 'Bearish'
Symbol = 'EURUSD'

In [3]:
#This will need to be updated to take in the date and time from previous script
if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
        quit()
mt5.login()
timezone = pytz.timezone("UTC")
now = datetime.now(timezone)
start = datetime.now(timezone) - dt.timedelta(days=5)
utc_from = datetime(start.year, start.month, start.day)
utc_to = datetime(now.year, now.month, now.day, now.hour, now.minute, now.second)
rates = mt5.copy_rates_range(Symbol, mt5.TIMEFRAME_M15, utc_from, utc_to)
htf = pd.DataFrame(rates)
htf['time']=pd.to_datetime(htf['time'], unit='s')
start=time.time()

In [15]:
for i in range(2,htf.shape[0]):
    current = htf.iloc[i,:]
    prev = htf.iloc[i-1,:]
    prev_2 = htf.iloc[i-2,:]
    prev_3 = htf.iloc[i-3,:]
    realbody = abs(current['open'] - current['close'])
    candle_range = current['high'] - current['low']
    idx = htf.index[i]
    htf.loc[idx,'Bullish FVG'] = current['low'] > prev_2['high'] and current['high'] > prev['high'] and prev['high'] > prev_2['high']
    htf.loc[idx,'Bearish FVG'] = current['high'] < prev_2['low'] and current['low'] < prev['low'] and prev['low'] < prev_2['low']
    if htf.loc[idx,'Bullish FVG'] == True:
        htf.loc[idx,'PEP'] = htf.loc[idx, 'low']
        if htf.loc[idx, 'low'] > 1.003:
            Bull_PEP= htf.loc[idx, 'low']
            Bull_SL=htf.loc[idx-2, 'low']
            PEP_time=htf.loc[idx, 'time']
    elif htf.loc[idx,'Bearish FVG'] == True:
        htf.loc[idx,'PEP'] = htf.loc[idx, 'high']
        if htf.loc[idx, 'high'] > 1.004:
            Bear_PEP= htf.loc[idx, 'high']
            Bear_SL=htf.loc[idx-2, 'high']
            PEP2_time=htf.loc[idx, 'time']

In [16]:
Bear_PEP

1.00413

In [11]:
htf.tail(15)

,time,open,high,low,close,tick_volume,spread,real_volume,Bullish FVG,Bearish FVG,PEP
449,2022-08-19 20:15:00,1.00433,1.00454,1.00405,1.00415,562,0,0,True,False,1.00405
450,2022-08-19 20:30:00,1.00414,1.00419,1.00369,1.00408,543,0,0,False,False,NaN
451,2022-08-19 20:45:00,1.00408,1.00449,1.00405,1.00434,609,0,0,False,False,NaN
452,2022-08-19 21:00:00,1.00434,1.00434,1.00353,1.00356,619,0,0,False,False,NaN
453,2022-08-19 21:15:00,1.00354,1.00440,1.00353,1.00418,601,0,0,False,False,NaN
454,2022-08-19 21:30:00,1.00419,1.00460,1.00384,1.00396,559,0,0,False,False,NaN
455,2022-08-19 21:45:00,1.00396,1.00397,1.00322,1.00330,608,0,0,False,False,NaN
456,2022-08-19 22:00:00,1.00333,1.00362,1.00321,1.00356,694,0,0,False,True,1.00362
457,2022-08-19 22:15:00,1.00356,1.00375,1.00346,1.00359,499,0,0,False,False,NaN
458,2022-08-19 22:30:00,1.00359,1.00390,1.00359,1.00384,548,0,0,False,False,NaN


In [7]:
if MS == 'Bullish':
    print (PEP_time)
    a = (round(Bull_PEP,4)) 
    b = (round(Bull_SL,4))
    print ('PEP is:',a,' SL is:',a -.002, ' TP is:', a +.005)
elif MS == 'Bearish':
    print (PEP2_time)
    a = (round(Bear_PEP,4)) 
    b = (round(Bear_SL,4))
    print ('PEP is:',a,' SL is:',a +.002, ' TP is:', a -.005)

2022-08-19 22:00:00
PEP is: 1.0036  SL is: 1.0016  TP is: 1.0086


In [9]:
a = 4
high = 10
low = 1
high > a > low

True